In [21]:
#V1版本的不足之处;后续添加个检查单原子负载前后基底变化，从而智能化选择更加合适的吸附位
import numpy as np
import pandas as pd
import random
import os
from shutil import copy
from math import cos, sin, atan2, sqrt, pi ,radians, degrees

In [22]:
#类似20220822下选择的100个DMAZ这种文件夹底下需要有CONTCAR_*的文件

In [23]:
# 打开新文件
def open_poscar(name):
    f = open(name,'r+')
    flist= f.readlines()
    f.close()
    return flist

In [24]:
# 组成新文件数据
def get_Hads_poscar_data(flist, start_num, sample_type,  DMAZ_name):
    if sample_type == 'DMAZ':
        support_name = DMAZ_name.split('_')[-1]
        line1 = [str(x) for x in flist[0].split()]
        line2 = [str(x) for x in flist[1].split()]
        line3 = [str(x) for x in flist[2].split()]
        line4 = [str(x) for x in flist[3].split()]
        line5 = [str(x) for x in flist[4].split()]
        line6 = [str(x) for x in flist[5].split()] + ['H'] # 原子序号
        line7 = [str(x) for x in flist[6].split()] + ['1'] #原子个数
        line8 = [str(x) for x in flist[7].split()]
        num_atoms = sum([int(x) for x in flist[6].split()])
        if  str(line8).strip().upper()[2] == 'D':
            x_cartesian = []
            y_cartesian = []
            z_cartesian = []
            for i in range(start_num, num_atoms + start_num): 
    #         x_cartesian.append(float(line[i].split()[0]) * a1 + float(line[i].split()[1]) * a2 + float(line[i].split()[2]) * a3)
    #         y_cartesian.append(float(line[i].split()[0]) * b1 + float(line[i].split()[1]) * b2 + float(line[i].split()[2]) * b3)
    #         z_cartesian.append(float(line[i].split()[0]) * z1 + float(line[i].split()[1]) * z2 + float(line[i].split()[2]) * z3)
                x_cartesian.append(float(flist[i].split()[0]))
                y_cartesian.append(float(flist[i].split()[1]))
                z_cartesian.append(float(flist[i].split()[2]))
        cartesian_xyz_bare = list(zip(x_cartesian, y_cartesian, z_cartesian))
        atom_name_list = [str(x) for x in flist[5].split()]
        atom_num_list = [str(x) for x in flist[6].split()]
        c = [float(x) for x in flist[4].split()][-1]
        cartesian_xyz_H = get_cartesian_xyz(cartesian_xyz_bare, c, support_name, atom_name_list, atom_num_list)
        cartesian_xyz = cartesian_xyz_bare + cartesian_xyz_H
        new_list = []
        new_list.append(line1)
        new_list.append(line2)
        new_list.append(line3)
        new_list.append(line4)
        new_list.append(line5)
        new_list.append(line6)
        new_list.append(line7)
        new_list.append(line8)
        for i in range(len(cartesian_xyz)):
            new_list.append(cartesian_xyz[i])
    return new_list

In [25]:
def get_cartesian_xyz(cartesian_xyz_bare, c, support_name, atom_name_list, atom_num_list):
    atom_name_list = [str(x) for x in flist[5].split()]
    atom_num_list = [int(x) for x in flist[6].split()]
    MAZ_name = atom_name_list[:-1]
    if support_name in MAZ_name:
        support_num_index = sum(atom_num_list) -1
    else:
        support_index = atom_name_list.index(support_name)
        count = 0
        support_num_index = 0
        for i in range(len(atom_name_list)):
            if count < support_index:
                support_num_index += atom_num_list[i]
                count +=1
    distance_SH = get_radii_sum(support_name, 'H')
    cartesian_xyz_H = [(cartesian_xyz_bare[support_num_index][0], 
                        cartesian_xyz_bare[support_num_index][1],
                        cartesian_xyz_bare[support_num_index][2]+distance_SH/c)]
    return cartesian_xyz_H

In [26]:
# 获取原子半径和
def get_radii_sum(atom1, atom2):
    # 获取涉及到的原子半径
    data = pd.read_excel('共价半径.xlsx', header=None)
    # 生成原子半径字典
    data_radius = dict_radius_production(data)
    radii_sum = data_radius[atom1] + data_radius[atom2]
    return radii_sum

In [27]:
# 生成共价半径字典
def dict_radius_production(data):
    dict_radius = {}
    for i in range(len(data)):
        dict_radius.update({data.iloc[i,0]:data.iloc[i,1]})
    return dict_radius

In [28]:
# 写入新文件
def write_newposcar(path, element_name, new_poscar):
    new_poscar_name = path + '/POSCAR_' + ''.join(element_name)
    f2=open(new_poscar_name,"w")
    for i in range(len(new_poscar)):
        s1 = str(new_poscar[i]).replace('[','').replace(']','').replace('(','').replace(')','')
        s2 = s1.replace("'",'').replace(',','') + '\n' 
        f2.write(s2)
    f2.close()

In [29]:
# 生成DMAZ的HER结构
Num_select = 100  #选择个数
name = '20220822' #命名，以时间和英文为命名方式
name_total = name + '下选择的' + str(Num_select) + '个DMAZ'
for i in os.listdir(name_total):
    os.makedirs(name_total+'_HER/'+i)
    from_path = name_total+ '/' + i + '/CONTCAR_' + i
    to_path = name_total+'_HER/'+ i + '/CONTCAR_' + i
    copy(from_path, to_path)
start_num = 9
for i in os.listdir(name_total+'_HER'):
    path = name_total+'_HER/'+ i
    flist = open_poscar(name_total+'_HER/'+ i + '/CONTCAR_' + i)
    new_list = get_Hads_poscar_data(flist, start_num -1 , 'DMAZ', i)
    write_newposcar(path, i, new_list) 
#     to_path_II = name+'_HER/'+ '/CONTCAR_' + i + '_' + ads_site
#     to_path_II2 = ''.join(to_path_II)
#     copy(from_path2, to_path_II2)

In [177]:
# 100select中Zr_C_N_W Zr_C_N_Co Pt_Si_As_Mn Mn_Sn_N_Hf Ir_Sn_As_W Ir_C_N_Y Fe_C_P_Fe这些结构有问题